<h1><center>Laboratorio 8: Aprendizaje Supervisado 🔮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla y Ignacio Meza
- Auxiliar: Sebastián Tinoco
- Ayudante: Felipe Arias y Diego Cortez

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Ricardo Espinoza
- Nombre de alumno 2: Cristóbal Guerra


### **Link de repositorio de GitHub:** `https://github.com/cristobalg68/Labs-Espinoza-Guerra.git`

## Temas a tratar

- Aprendizaje Supervisado
- Flujos de datos a través de `Pipelines`.

## Reglas

- **Grupos de 2 personas**
- Asistencia **obligatoria** a instrucciones del lab (viernes 16.15). Luego, pueden quedarse trabajando en las salas o irse.
- **No se revisarán entregas de personas ausentes**. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

## Objetivos principales del laboratorio

- Comprender el funcionamiento de clasificadores/regresores.
- Generar múltiples modelos predictivos.
- Comprender las ventajas de crear modelos en pipeline vs hacer las operaciones a mano.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre `DataFrames`.

### Importamos librerias utiles 😸

In [138]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn import set_config

# Clasificación
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import svm
from sklearn import linear_model

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import r2_score

#Libreria para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


https://www.kaggle.com/antoinekrajnc/soccer-players-statistics

# 1. Predicciones Futboleras

<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=1BnO4tyh3vM2P199Ec9s3JjngQ4qQ9seP"
" width="300">
</p>


Tras el trágico despido de la mítica mascota de Maipú, Renacín decide adentrarse como consultor en el mercado futbolero, el cuál (para variar...) está cargado en especulaciones. 

Como su principal tarea será asesorar a los directivos de los clubes sobre cuál jugador comprar y cuál no, Renacín desea generar modelos predictivos que evaluén distintas características de los jugadores; todo con el fin de tomar decisiones concretas basadas en los datos.

Sin embargo, su condición de corporeo le impidió tomar la versión anterior de MDS7202, por lo que este motivo Renacín contrata a su equipo para lograr su objetivo final. Dado que aún tiene fuertes vínculos con la dirección de deportes de la municipalidad, el corporeo le entrega base de datos con las estadísticas de cada jugador para que su equipo empieze a trabajar ya con un dataset listo para ser usado.


**Los Datos**

Para este laboratorio deberán trabajar con los csv `statsplayers.csv` y `salarios.pickle`, donde deberán aplicar algoritmos de de aprendizaje supervisado (clasificación y regresión) en base a características que describen de jugadores de futbol.

Para comenzar cargue el dataset señalado y a continuación vea el reporte **`Player_Stats_Report.html`** (adjunto en la carpeta del enunciado) que describe las características principales del `DataFrame`.

## 1.0 Lectura del dataset

In [139]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [140]:
# path_players = '/content/drive/Othercomputers/Mi PC/Estudios/Universidad/Semestre_2023_1/Laboratorio_de_Programación_Científica_para_Ciencia_de_Datos/Laboratorios/Lab_8/stats_players.csv'
path_players = '/content/drive/My Drive/Lab_8/stats_players.csv'

In [141]:
df_players = pd.read_csv(path_players)
df_players

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,32,High / Low,4,...,90,95,85,92,93,90,81,76,85,88
1,Lionel Messi,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,...,90,68,71,85,95,88,89,90,74,85
2,Neymar,Brazil,LW,LW,174,68,Right,25,High / Medium,5,...,96,61,62,78,89,77,79,84,81,83
3,Luis Suárez,Uruguay,LS,ST,182,85,Right,30,High / Medium,4,...,86,69,77,87,94,86,86,84,85,88
4,Manuel Neuer,Germany,GK,GK,193,92,Right,31,Medium / Medium,4,...,52,78,25,25,13,16,14,11,47,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17583,Adam Dunbar,Republic of Ireland,NaN,Sub,183,82,Right,19,Medium / Medium,1,...,27,56,14,16,13,13,11,13,15,12
17584,Dylan McGoey,Republic of Ireland,NaN,Sub,185,80,Right,19,Medium / Medium,2,...,28,53,12,17,12,11,12,13,16,12
17585,Tommy Ouldridge,England,NaN,Res,173,61,Right,18,High / Medium,2,...,54,61,41,44,28,42,35,36,42,37
17586,Mark Foden,Scotland,NaN,Sub,180,80,Right,21,Medium / Medium,3,...,34,48,15,23,14,12,13,12,24,12


## 1.1 Predicción de Seleccionados Nacionales 

Como primera tarea, Renacín, intrigado por la posibilidad de saber qué tan reconocido es un jugador, le consulta a su equipo si es posible predecir si un jugador será o no seleccionado nacional a partir de sus estadísticas en el juego. 

<p align="center">
  <img src="https://thumbs.gfycat.com/ThornyFrankFoxhound-size_restricted.gif" width="500">
</p>

### 1.1.1 Generación de Labels para la Clasificación [Sin Puntaje]


Primero comience generando las labels para la clasificación. Para esto, trabaje sobre el atributo ```National_Position``` suponiendo que los valores nulos son jugadores no seleccionados para representar a su país. 

Hecho esto, ¿Cuantos ejemplos por cada clase se tienen? Comente lo que observa.



**Respuesta:**

In [142]:
df_players['National_Position'] = df_players['National_Position'].fillna(0)
df_players.loc[df_players['National_Position'] != 0, 'National_Position'] = 1

In [143]:
df_players['National_Position'].value_counts()

0    16513
1     1075
Name: National_Position, dtype: int64

Al ver la cantidad de elementos por cada clase se puede concluir que el dataset se encuentra desbalanceado. Notar qué mayor parte de registros corresponden a jugadores de la clase no seleccionados.

### 1.1.2 Camino a la clasificación [1 punto]

<p align="center">
  <img src="https://c.tenor.com/SoXZJHWnhKAAAAAC/pitbull-medel.gif" width="500">
</p>

Para preprocesar el dataset, genere un `ColumnTransformer` en donde especifique las transformaciones que hay que realizar para cada columna (por ejemplo StandarScaler, MinMaxScaler, OneHotEncoder, etc...) para que puedan ser utilizadas correctamente por el modelo predictivo y guardelo en algúna variable.

Luego, comente y justifique las transformaciones elegidas sobre cada una de las variables (para esto utilice el material ```Player_Stats_Report.html``` que viene en el zip del lab), al igual que las transformaciones aplicadas.

Hecho lo anterior, **defina al menos 3 pipelines para la clasificación**, en donde utilice el mismo ColumnTransformer definido anteriormente, pero que varie entre cada pipeline los clasificadores. 

Para seleccionar los clasificadores más adecuados, utilice la siguiente guía:

<p align="center">
  <img src="https://scikit-learn.org/stable/_static/ml_map.png" width="500">
</p>

<br/>

Con ella, comente y justifique cada una de las decisiones tomadas al momento de desarrollar su pipeline.

**Nota:** Si tiene problemas al utilizar OneHotEncoder puede utilizar el parámetro handle_unknown='ignore'. Esto hace que en la codificación se omitan las categorias que no aparecen en el entrenamiento. Pregunta dudosa (no tiene puntaje), ¿esto tiene sentido a nivel de modelos?.

**To-Do:**

- [X] Genere un ```ColumnTransformer``` enfocado en preprocesar los datos.
- [X] Indicar y Justificar que preprocesamiento utiliza sobre cada columna.
- [X] Crear 3 ``pipelines`` con diferentes clasificadores.
- [X] Para seleccionar los clasificadores base sus decisiones en la siguiente [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)
- [X] No entrenar los ``pipelines`` aún.

**Nota:** No es necesario entrenar los clasificadores aún.

**Respuesta:**

In [144]:
set_config(transform_output="pandas")

categorical_transformer = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
categorical_columns = ['Nationality', 'Club_Position']

estandar_scaler = StandardScaler()
scaler_columns = ['Height', 'Weight','Age', 'Aggression', 'Reactions', 'Vision', 'Composure', 'Long_Pass', 'Acceleration', 'Speed', 'Strength', 'Balance', 'Agility', 'Jumping', 'Finishing', 'Volleys']

min_max = MinMaxScaler()
minmax_columns = ['Ball_Control', 'Dribbling', 'Marking', 'Sliding_Tackle', 'Standing_Tackle', 'Interceptions', 'Crossing', 'Short_Pass', 'Stamina', 'Heading', 'Shot_Power', 'Long_Shots', 'Curve', 'Freekick_Accuracy', 'Penalties']

ordinal_transformer = OrdinalEncoder(categories=[['Right', 'Left'], ['Low / Low', 'Low / Medium', 'Low / High', 'Medium / Low', 'Medium / Medium', 'Medium / High', 'High / Low', 'High / Medium', 'High / High']])
ordinal_columns = ['Preffered_Foot', 'Work_Rate']

preprocessor = ColumnTransformer(
    transformers=[('Categorical', categorical_transformer, categorical_columns),
                  ('Ordinal', ordinal_transformer, ordinal_columns),
                  ('Standard Scaler', estandar_scaler, scaler_columns),
                  ('MinMax Scaler', min_max, minmax_columns)
                  ])

El procesamiento que se le aplicó a cada columna se dividió en 6 tipos:

  * Se eliminó la columna 'Name', pues esta no debería importar a la hora de clasificar.

  * No se trasforman las columnas 'Weak_foot', 'Skill_Moves' y 'National_Position', pues estas ya se encuentran lista para ser utilizadas.

  * Para las columnas 'Nationality' y 'Club_Position' se utilizó un OneHotEncoder, pues estas son columnas categóricas sin orden.

  * Para las columnas 'Preffered_Foot' y 'Work_Rate' se utilizó un OrdinalEncoder, pues estas son columnas categóricas con un orden.

  * Para las columnas 'Ball_Control', 'Dribbling', 'Marking', 'Sliding_Tackle', 'Standing_Tackle', 'Interceptions', 'Crossing', 'Short_Pass', 'Stamina', 'Heading', 'Shot_Power', 'Long_Shots', 'Curve', 'Freekick_Accuracy' y 'Penalties' se utilizó un MinMaxScaler, pues estas columnas no distribuyen normal.

  * Para las columnas 'Height', 'Weight','Age', 'Aggression', 'Reactions', 'Vision', 'Composure', 'Long_Pass', 'Acceleration', 'Speed', 'Strength', 'Balance', 'Agility', 'Jumping', 'Finishing' y 'Volleys' se utilizó un StandardScaler, pues estas columnas distribuyen normal.

In [145]:
Pipe1 = Pipeline([
            ('Se trasforman las columnas', preprocessor),
            ('SVC', svm.SVC(kernel='linear', degree=3, gamma='auto'))
        ])

Pipe2 = Pipeline([
            ('Se trasforman las columnas', preprocessor),
            ('AdaBoostClassifier', AdaBoostClassifier(n_estimators=100))
        ])

Pipe3 = Pipeline([
            ('Se trasforman las columnas', preprocessor),
            ('Random Forest Classifier', RandomForestClassifier(n_estimators=100, max_depth=3, random_state=0))
        ])

### 1.1.3 Entrenemos los pipelines [1 punto]

<p align="center">
  <img src="https://i.gifer.com/7Cc8.gif" width="350">
</p>


Ahora, entrene los `pipeline` generados en los pasos anteriores. Para esto, primero separe los datos de entrenamiento en un conjunto de entrenamiento y de prueba  (la proporción queda a su juicio).

En este paso, seleccione los ejemplos de forma aleatoria e intente mantener la distribución original de labels de cada clase en los conjuntos de prueba/entrenamiento. (vea la documentación de `train_test_split`).


Luego, entrene los pipelines

Una vez entrenado su modelo, evalue su rendimiento a través de diferentes métricas, comentando que significa cada uno de los valores obtenidos. Puede usar la función `classification_report` para corroborar sus resultados.

- ¿Qué implican los valores de accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional)?
- ¿Podrían mejorarse los resultados?, ¿Cómo?
- ¿Influye la cantidad de ejemplos por clase?

**To-Do**:

- [X] Separar el conjunto de datos en entrenamiento y de prueba.
- [X] Entrenar los `pipelines`.
- [X] Utilizar las métricas para evaluar los modelos generados.


**Respuesta:**

In [146]:
# df_players = df_players.drop(columns=['Name'])
X = df_players.drop(columns=['National_Position'])
y = df_players['National_Position'].astype('int')

In [147]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=22)

In [148]:
unique, count = np.unique(y_train, return_counts=True)
print(f'Existen {count[0]} de la clase {unique[0]} y {count[1]} de la clase {unique[1]} en el conjunto de entrenamiento')
unique, count = np.unique(y_test, return_counts=True)
print(f'Existen {count[0]} de la clase {unique[0]} y {count[1]} de la clase {unique[1]} en el conjunto de prueba')

Existen 11564 de la clase 0 y 747 de la clase 1 en el conjunto de entrenamiento
Existen 4949 de la clase 0 y 328 de la clase 1 en el conjunto de prueba


In [149]:
Pipe1.fit(X_train, y_train)
Pipe2.fit(X_train, y_train)
Pipe3.fit(X_train, y_train)

Pipeline(steps=[('Se trasforman las columnas',
                 ColumnTransformer(transformers=[('Categorical',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['Nationality',
                                                   'Club_Position']),
                                                 ('Ordinal',
                                                  OrdinalEncoder(categories=[['Right',
                                                                              'Left'],
                                                                             ['Low '
                                                                              '/ '
                                                                              'Low',
                                                                              'Low '
                                                                              '/ '
                                                                              'Medium',
                                                                              'Low '
                                                                              '/ '
                                                                              'High',
                                                                              'Medium '
                                                                              '/ '
                                                                              'Low',
                                                                              'Medium '
                                                                              '/ '
                                                                              'Medium',
                                                                              'Medium '
                                                                              '/ '
                                                                              'High',
                                                                              'High '
                                                                              '/ '
                                                                              'L...
                                                   'Balance', 'Agility',
                                                   'Jumping', 'Finishing',
                                                   'Volleys']),
                                                 ('MinMax Scaler',
                                                  MinMaxScaler(),
                                                  ['Ball_Control', 'Dribbling',
                                                   'Marking', 'Sliding_Tackle',
                                                   'Standing_Tackle',
                                                   'Interceptions', 'Crossing',
                                                   'Short_Pass', 'Stamina',
                                                   'Heading', 'Shot_Power',
                                                   'Long_Shots', 'Curve',
                                                   'Freekick_Accuracy',
                                                   'Penalties'])])),
                ('Random Forest Classifier',
                 RandomForestClassifier(max_depth=3, random_state=0))])

In [150]:
y_pred_1 = Pipe1.predict(X_test)
y_pred_2 = Pipe2.predict(X_test)
y_pred_3 = Pipe3.predict(X_test)

print(f'El accuracy del Pipeline 1 es {accuracy_score(y_test, y_pred_1)}')
print(f'El accuracy del Pipeline 2 es {accuracy_score(y_test, y_pred_2)}')
print(f'El accuracy del Pipeline 3 es {accuracy_score(y_test, y_pred_3)}')

print(f'El precision del Pipeline 1 es {precision_score(y_test, y_pred_1)}')
print(f'El precision del Pipeline 2 es {precision_score(y_test, y_pred_2)}')
print(f'El precision del Pipeline 3 es {precision_score(y_test, y_pred_3)}')

print(f'El recall del Pipeline 1 es {recall_score(y_test, y_pred_1)}')
print(f'El recall del Pipeline 2 es {recall_score(y_test, y_pred_2)}')
print(f'El recall del Pipeline 3 es {recall_score(y_test, y_pred_3)}')

target_names = ['NOTSELECTED', 'SELECTED']

print(classification_report(y_test, y_pred_1, target_names=target_names))
print(classification_report(y_test, y_pred_2, target_names=target_names))
print(classification_report(y_test, y_pred_3, target_names=target_names))

El accuracy del Pipeline 1 es 0.9456130377108205
El accuracy del Pipeline 2 es 0.9509190828122039
El accuracy del Pipeline 3 es 0.9378434716695092
El precision del Pipeline 1 es 0.6990291262135923
El precision del Pipeline 2 es 0.6682926829268293
El precision del Pipeline 3 es 0.0
El recall del Pipeline 1 es 0.21951219512195122
El recall del Pipeline 2 es 0.4176829268292683
El recall del Pipeline 3 es 0.0
              precision    recall  f1-score   support

 NOTSELECTED       0.95      0.99      0.97      4949
    SELECTED       0.70      0.22      0.33       328

    accuracy                           0.95      5277
   macro avg       0.82      0.61      0.65      5277
weighted avg       0.93      0.95      0.93      5277

              precision    recall  f1-score   support

 NOTSELECTED       0.96      0.99      0.97      4949
    SELECTED       0.67      0.42      0.51       328

    accuracy                           0.95      5277
   macro avg       0.82      0.70      0.74   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: Unde

**Respuesta**:

* Accuracy: Se observan resultados muy buenos, entre 0.93 y 0.95 para todos los clasificadores. Dado que corresponde a la proporción de predicciones correctas (positivas y negativas) con respecto al total, no dice mucho con respecto al rendimiento del modelo sobre la clase positiva, ya que ambas clases están muy desbalanceadas, por lo que el accuracy puede ser alto a pesar de no estar identificando correctamente esos casos.

* Precisión: Se observan resultados de 0, 0.67 y 0.7 para 'Random Forest Classifier', 'AdaBoostClassifier' y 'Linear SVC' respectivamente. Dado que corresponde a la proporción de predicciones positivas correctas con respecto a todas las predicciones positivas hechas, indica el rendimiento del modelo al momento de predecir correctamente a los jugadores seleccionados nacionales (clase positiva). Por ello, se dice que el 'Random Forest Classifier' falla en todas sus predicciones de la clase positiva, mientras que 'AdaBoostClassifier' y 'Linear SVC' aciertan aproximadamente 2/3 de sus predicciones positivas totales.

* Recall: Se observan resultados de 0, 0.42 y 0.22 para 'Random Forest Classifier', 'AdaBoostClassifier' y 'Linear SVC' respectivamente. Dado que corresponde a la proporción de predicciones positivas correctas con respecto a todos los casos positivos reales, indica qué tan bueno es el modelo para identificar a todos los jugadores seleccionados nacionales (clase positiva). Por ello, se dice que en general todos los clasificadores probados tienen un rendimiento muy malo, ya que logran identificar menos del 50% de los casos positivos, peor que el rendimiento esperado de lanzar una moneda al aire para clasificar.

De los puntos anteriores se concluye que los tres clasificadores están sesgados hacia la clase negativa (deducible a partir del alto valor de accuracy) e identifican menos de la mitad de los casos positivos reales, lo cual es malo. Pero 'AdaBoostClassifier' y 'Linear SVC' lo hacen acertando aproximadamente 2/3 del total de veces que predicen que un dato pertenece a la clase positiva.

Los resultados anteriores podrían mejorar con los siguientes procedimientos:

* Aplicar alguna técnica de balanceo de clases como undersampling u oversampling, para evitar sesgo hacia la clase negativa (No seleccionado) que es predominante.

* Seleccionar y utilizar variables más relevantes del dataset, para reducir el número de características que no aportan información para el problema en cuestión y pueden estar afectando negativamente el rendimiento del modelo. 

* Experimentar ajustando los hiperparámetros de los modelos.

Con respecto a la influencia que tiene la cantidad de ejemplos sobre el rendimiento de los modelos para predecir la clase positiva, se dice que sí puede influir ya que el modelo puede sesgarse hacia la clase predominante (negativa en este problema) y no generalizar correctamente, resultando en un bajo rendimiento hacia la clase minoritaria (positiva en este problema).

## 1.2 Predicción de posiciones de jugadores [2 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/E1rfA1aWEAYU6Ny.jpg" width="500">
</p>

En una nueva jornada de desmesuradas transacciones deportivas, Renacín escuchó a sus colegas discutir acerca de que el precio de cada jugador depende en gran medida de la posición en la cancha en la que juega. Y además, que hay bastantes jugadores nuevos que no tienen muy claro en que posición verdaderamente brillarían, por lo que actualmente puede que actualmente estén jugando en posiciones sub-optimas. 

Viendo que los resultados del primer análisis no son tan esperanzadores, el corporeo los comanda a cambiar su tarea: ahora, les solicita que construyan un clasificador enfocado en predecir la mejor posición de los jugadores en la cancha según sus características. 

Para lograr esto, primero, les pide que etiqueten de la siguiente manera los valores que aparecen en el atributo ```Club_Position```, pidiendo que agrupen los valores en los siguientes grupos:

**Nota**:  Renacín les recalca que **no deben utilizar los valores ```Sub``` y ```Res``` de esta columna**.

```python
ataque = ['ST', 'CF'] 
central_ataque = ['RW', 'CAM', 'LW'] 
central = ['RM', 'CM', 'LM'] 
central_defensa = ['RWB', 'CDM', 'LWB']
defensa = ['RB', 'CB', 'LB']
arquero = ['GK']
```

Cabe señalar que al igual como lo realizado con la clasificación binaria, deberá justificar en base a la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) la elección del clasificador y se deben comentar los resultados obtenidos en la clasificación.

**Nota:** Clasifique solamente con las clases señaladas, si observa mas clases eliminelas de la clasficación.

**To-Do**:

- [X] Aplique las etiquetas descritas anteriormente en cada uno de los valores señalados en esta sección.
- [X] Cuente cuantos por clase quedan.
- [X] Entrene el nuevo pipeline y ejecute una evaluación de este.  
- [X] Comente los resultados obtenidos.

**Respuesta:**

In [151]:
df_players = df_players.dropna(subset=['Club_Position'])
df_players = df_players[~df_players['Club_Position'].isin(['RCM', 'LCB', 'LCM', 'RS', 'RCB', 'LDM', 'RDM', 'LF', 'LAM', 'LS', 'RF', 'RAM', 'Sub', 'Res'])]

class_id = {'ST': 0,
            'CF': 0,
            'RW': 1,
            'CAM': 1,
            'LW': 1,
            'RM': 2,
            'CM': 2,
            'LM': 2,
            'RWB': 3,
            'CDM': 3,
            'LWB': 3,
            'RB': 4,
            'CB': 4,
            'LB': 4,
            'GK': 5}

df_players['New_Club_Position'] = df_players['Club_Position'].map(class_id)

In [152]:
df_players['New_Club_Position'].value_counts()

4    1180
2     907
5     632
1     581
0     430
3     209
Name: New_Club_Position, dtype: int64

In [153]:
set_config(transform_output="pandas")

categorical_transformer = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
categorical_columns = ['Nationality']

estandar_scaler = StandardScaler()
scaler_columns = ['Height', 'Weight','Age', 'Aggression', 'Reactions', 'Vision', 'Composure', 'Long_Pass', 'Acceleration', 'Speed', 'Strength', 'Balance', 'Agility', 'Jumping', 'Finishing', 'Volleys']

min_max = MinMaxScaler()
minmax_columns = ['Ball_Control', 'Dribbling', 'Marking', 'Sliding_Tackle', 'Standing_Tackle', 'Interceptions', 'Crossing', 'Short_Pass', 'Stamina', 'Heading', 'Shot_Power', 'Long_Shots', 'Curve', 'Freekick_Accuracy', 'Penalties']

ordinal_transformer = OrdinalEncoder(categories=[['Right', 'Left'], ['Low / Low', 'Low / Medium', 'Low / High', 'Medium / Low', 'Medium / Medium', 'Medium / High', 'High / Low', 'High / Medium', 'High / High']])
ordinal_columns = ['Preffered_Foot', 'Work_Rate']

preprocessor = ColumnTransformer(
    transformers=[('Categorical', categorical_transformer, categorical_columns),
                  ('Ordinal', ordinal_transformer, ordinal_columns),
                  ('Standard Scaler', estandar_scaler, scaler_columns),
                  ('MinMax Scaler', min_max, minmax_columns)
                  ])

In [154]:
Pipe4 = Pipeline([
            ('Se trasforman las columnas', preprocessor),
            ('SVC', svm.SVC(kernel='linear', degree=3, gamma='auto'))
        ])

In [155]:
df_players = df_players.drop(columns=['National_Position', 'Club_Position'])
X = df_players.drop(columns=['New_Club_Position'])
y = df_players['New_Club_Position'].astype('int')

In [156]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=22)

In [157]:
unique_tr, count_tr = np.unique(y_train, return_counts=True)
unique_te, count_te = np.unique(y_test, return_counts=True)
for i in range(5):
    print(f'Existen {count_tr[i]} de la clase {unique_tr[i]} en el conjunto de entrenamiento')
    print(f'Existen {count_te[i]} de la clase {unique_te[i]} en el conjunto de prueba')

Existen 317 de la clase 0 en el conjunto de entrenamiento
Existen 113 de la clase 0 en el conjunto de prueba
Existen 404 de la clase 1 en el conjunto de entrenamiento
Existen 177 de la clase 1 en el conjunto de prueba
Existen 632 de la clase 2 en el conjunto de entrenamiento
Existen 275 de la clase 2 en el conjunto de prueba
Existen 141 de la clase 3 en el conjunto de entrenamiento
Existen 68 de la clase 3 en el conjunto de prueba
Existen 836 de la clase 4 en el conjunto de entrenamiento
Existen 344 de la clase 4 en el conjunto de prueba


In [158]:
Pipe4.fit(X_train, y_train)

Pipeline(steps=[('Se trasforman las columnas',
                 ColumnTransformer(transformers=[('Categorical',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['Nationality']),
                                                 ('Ordinal',
                                                  OrdinalEncoder(categories=[['Right',
                                                                              'Left'],
                                                                             ['Low '
                                                                              '/ '
                                                                              'Low',
                                                                              'Low '
                                                                              '/ '
                                                                              'Medium',
                                                                              'Low '
                                                                              '/ '
                                                                              'High',
                                                                              'Medium '
                                                                              '/ '
                                                                              'Low',
                                                                              'Medium '
                                                                              '/ '
                                                                              'Medium',
                                                                              'Medium '
                                                                              '/ '
                                                                              'High',
                                                                              'High '
                                                                              '/ '
                                                                              'Low',
                                                                              'High '
                                                                              '/ '
                                                                              'Me...
                                                   'Speed', 'Strength',
                                                   'Balance', 'Agility',
                                                   'Jumping', 'Finishing',
                                                   'Volleys']),
                                                 ('MinMax Scaler',
                                                  MinMaxScaler(),
                                                  ['Ball_Control', 'Dribbling',
                                                   'Marking', 'Sliding_Tackle',
                                                   'Standing_Tackle',
                                                   'Interceptions', 'Crossing',
                                                   'Short_Pass', 'Stamina',
                                                   'Heading', 'Shot_Power',
                                                   'Long_Shots', 'Curve',
                                                   'Freekick_Accuracy',
                                                   'Penalties'])])),
                ('SVC', SVC(gamma='auto', kernel='linear'))])

In [159]:
y_pred_4 = Pipe4.predict(X_test)

print(f'El accuracy del Pipeline 4 es {accuracy_score(y_test, y_pred_4)}')

print(f'El precision del Pipeline 4 es {precision_score(y_test, y_pred_4, average=None)}')

print(f'El recall del Pipeline 4 es {recall_score(y_test, y_pred_4, average=None)}')

target_names = ['ataque', 'central_ataque', 'central', 'central_defensa', 'defensa', 'arquero']

print(classification_report(y_test, y_pred_4, target_names=target_names))

El accuracy del Pipeline 4 es 0.7461928934010152
El precision del Pipeline 4 es [0.71212121 0.56034483 0.5979021  0.65384615 0.79186603 1.        ]
El recall del Pipeline 4 es [0.83185841 0.36723164 0.62181818 0.25       0.9622093  0.99512195]
                 precision    recall  f1-score   support

         ataque       0.71      0.83      0.77       113
 central_ataque       0.56      0.37      0.44       177
        central       0.60      0.62      0.61       275
central_defensa       0.65      0.25      0.36        68
        defensa       0.79      0.96      0.87       344
        arquero       1.00      1.00      1.00       205

       accuracy                           0.75      1182
      macro avg       0.72      0.67      0.67      1182
   weighted avg       0.73      0.75      0.73      1182



**Respuesta**:

Sobre los resultados obtenidos, primero se observa que los 'support' de cada clase son muy pequeños, lo cual es preocupante ya que los resultados obtenidos pueden no ser muy generalizables. Luego, también se nota que el accuracy del 'Linear SVC' es de 0.75, lo cual es aceptable. Asimismo, los precision son mayores a 0.65 para cuatro de las seis clases (ataque, central_defensa, defensa y arquero), lo cual es aceptable ya que es comparable a acertar aproximadamente 2/3 de las veces que predice para esa posición. Las otras dos clases (central_ataque y 
central) tienen valores menores o iguales a 0.60, lo cual se considera bajo ya que falla poco menos de la mitad de las predicciones positivas que realiza para esas clases. Por último, los recall son mayores a 0.83 para tres de las seis clases (ataque, defensa y arquero), lo cual es bastante bueno ya que predice correctamente la mayoría de los casos positivos reales para esas clases, mientras que para 'central' tiene un rendimiento decente con 0.62 y para 'central_ataque' y 'central_defensa' tiene rendimientos extremadamente malos (0.37 y 0.25 respectivamente).

Quizás sería posible mejorar estos resultados si se consideran las posiciones 'RCM', 'LCB', 'LCM', 'RS', 'RCB', 'LDM', 'RDM', 'LF', 'LAM', 'LS', 'RF' y 'RAM' dentro del diccionario que crea las nuevas posiciones ('ataque', 'central_ataque', 'central', 'central_defensa', 'defensa', 'arquero'), esto porque en aquellas posiciones no consideradas hay muchos jugadores que se eliminaron, lo cual disminuye de inmediato el rendimiento del modelo. También se podría experimentar ajustando los hiperparámetros de los modelos para intentar conseguir mejor rendimiento en general.


## 1.3 Predicción de Sueldos [2 puntos]

<p align="center">
  <img src="https://media.giphy.com/media/3oEjHWtHAFcOKS4iA0/giphy.gif" width="460">
</p>

Queriendo ahondar aún más en el mercado del balompíe, Renacin, logra obtener (de una manera no muy formal) los sueldos de múltiples futbolistas y los guarda en el archivo ```sueldos.csv```. Con ellos les solicita que generen un regresor que les permita predecir el sueldo de los futbolistas en base a las características de los pichichis, esto, debido a su motivación por invertir y/o realizar especulación sobre los sueldos de jugadores.

Renacin es claro señalando que deben seguir utilizando la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) y comenten cada uno de los pasos realizados, para obtener su regresión lineal. Señalándoles que no aceptara un $R^2$ inferior a 0.35 para el modelo solicitado.

Para esta parte usted tiene total libertad en la generación del regresor, la unica exigencia es que utilice un pipeline para generar la regresión y utilice la metrica $R^2$ para medir el rendimiento de esta.

**To-Do**:

- [X] Explique en que consiste la métrica $R^2$
- [X] Generar un pipeline para la regresión.
- [X] Obtener un regresor con un $R^2$ superior a  $0.35$.
- [ ] Comente sus resultados y si es posible mejorar los resultados obtenidos. ¿Se necesitarían más datos o otros tipos de características o una combinación de ambos?

**Respuesta**

$R^2$ es una métrica que indica qué tan bien se ajusta el modelo generado a los datos observados, esta se utiliza para evaluar qué proporción de la variabilidad de la variable dependiente de un modelo (sueldo de un jugador para este problema) es explicada por las variables independientes del mismo. Toma valores entre 0 y 1, donde 0 implica que el modelo no explica ninguna variabilidad y 1 que el modelo se ajusta perfectamente a los datos.

In [160]:
path_sueldos = '/content/drive/My Drive/Lab_8/salarios.csv'

In [161]:
df_sueldos = pd.read_csv(path_sueldos)
df_sueldos = df_sueldos.drop(columns=['Unnamed: 0'])
df_sueldos = df_sueldos.rename(columns={'Player': 'Name'})
df_sueldos

,Name,Weekly Salary
0,Lionel Messi,1538905.0
1,Cristiano Ronaldo,1248536.0
2,Antoine Griezmann,997032.0
3,Neymar,797726.0
4,Gareth Bale,607706.0
...,...,...
2529,Simone Ghidotti,283.0
2530,Valentin Gendrey,100.0
2531,John Yeboah,70.0
2532,Antonis Aidonis,70.0


In [162]:
df_final = df_players.merge(df_sueldos, how='left')
df_final = df_final.dropna(subset=['Weekly Salary'])
df_final['Weekly Salary log'] = np.log10(df_final['Weekly Salary'])
df_final

,Name,Nationality,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,Skill_Moves,Ball_Control,...,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys,New_Club_Position,Weekly Salary,Weekly Salary log
0,Cristiano Ronaldo,Portugal,185,80,Right,32,High / Low,4,5,93,...,92,93,90,81,76,85,88,1,1248536.0,6.096401
1,Lionel Messi,Argentina,170,72,Left,29,Medium / Medium,4,4,95,...,85,95,88,89,90,74,85,1,1538905.0,6.187212
2,Neymar,Brazil,174,68,Right,25,High / Medium,5,5,95,...,78,89,77,79,84,81,83,1,797726.0,5.901854
3,Luis Suárez,Uruguay,182,85,Right,30,High / Medium,4,4,91,...,87,94,86,86,84,85,88,0,508923.0,5.706652
4,Manuel Neuer,Germany,193,92,Right,31,Medium / Medium,4,1,48,...,25,13,16,14,11,47,11,5,326233.0,5.513528
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3676,Dennis Lippert,Germany,172,63,Left,21,Medium / Medium,3,2,62,...,33,25,40,49,26,28,23,4,620.0,2.792392
3690,Charles Traoré,Mali,180,74,Left,25,Medium / Low,2,2,39,...,27,30,32,38,40,45,29,4,13289.0,4.123492
3704,Will Norris,England,191,76,Left,23,Medium / Medium,3,1,20,...,18,10,10,17,18,16,12,5,3958.0,3.597476
3842,Harvey Barnes,England,179,68,Left,19,Medium / Medium,3,3,59,...,57,50,52,56,52,49,50,1,13193.0,4.120344


In [163]:
set_config(transform_output="pandas")

categorical_transformer = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
categorical_columns = ['Nationality']

estandar_scaler = StandardScaler()
scaler_columns = ['Height', 'Weight','Age', 'Aggression', 'Reactions', 'Vision', 'Composure', 'Long_Pass', 'Acceleration', 'Speed', 'Strength', 'Balance', 'Agility', 'Jumping', 'Finishing', 'Volleys']

min_max = MinMaxScaler()
minmax_columns = ['Ball_Control', 'Dribbling', 'Marking', 'Sliding_Tackle', 'Standing_Tackle', 'Interceptions', 'Crossing', 'Short_Pass', 'Stamina', 'Heading', 'Shot_Power', 'Long_Shots', 'Curve', 'Freekick_Accuracy', 'Penalties']

ordinal_transformer = OrdinalEncoder(categories=[['Right', 'Left'], ['Low / Low', 'Low / Medium', 'Low / High', 'Medium / Low', 'Medium / Medium', 'Medium / High', 'High / Low', 'High / Medium', 'High / High']])
ordinal_columns = ['Preffered_Foot', 'Work_Rate']

preprocessor = ColumnTransformer(
    transformers=[('Categorical', categorical_transformer, categorical_columns),
                  ('Ordinal', ordinal_transformer, ordinal_columns),
                  ('Standard Scaler', estandar_scaler, scaler_columns),
                  ('MinMax Scaler', min_max, minmax_columns)
                  ])

In [164]:
Pipe5 = Pipeline([
            ('Se trasforman las columnas', preprocessor),
            ('Ridge', linear_model.Ridge(alpha=.5))
        ])

In [165]:
X = df_final.drop(columns=['Weekly Salary log', 'Weekly Salary'])
y = df_final['Weekly Salary log']

In [166]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=22)

In [167]:
Pipe5.fit(X_train, y_train)

Pipeline(steps=[('Se trasforman las columnas',
                 ColumnTransformer(transformers=[('Categorical',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['Nationality']),
                                                 ('Ordinal',
                                                  OrdinalEncoder(categories=[['Right',
                                                                              'Left'],
                                                                             ['Low '
                                                                              '/ '
                                                                              'Low',
                                                                              'Low '
                                                                              '/ '
                                                                              'Medium',
                                                                              'Low '
                                                                              '/ '
                                                                              'High',
                                                                              'Medium '
                                                                              '/ '
                                                                              'Low',
                                                                              'Medium '
                                                                              '/ '
                                                                              'Medium',
                                                                              'Medium '
                                                                              '/ '
                                                                              'High',
                                                                              'High '
                                                                              '/ '
                                                                              'Low',
                                                                              'High '
                                                                              '/ '
                                                                              'Me...
                                                   'Long_Pass', 'Acceleration',
                                                   'Speed', 'Strength',
                                                   'Balance', 'Agility',
                                                   'Jumping', 'Finishing',
                                                   'Volleys']),
                                                 ('MinMax Scaler',
                                                  MinMaxScaler(),
                                                  ['Ball_Control', 'Dribbling',
                                                   'Marking', 'Sliding_Tackle',
                                                   'Standing_Tackle',
                                                   'Interceptions', 'Crossing',
                                                   'Short_Pass', 'Stamina',
                                                   'Heading', 'Shot_Power',
                                                   'Long_Shots', 'Curve',
                                                   'Freekick_Accuracy',
                                                   'Penalties'])])),
                ('Ridge', Ridge(alpha=0.5))])

In [168]:
y_pred_5 = Pipe5.predict(X_test)

print(f'El R^2 del Pipeline 5 es {r2_score(y_test, y_pred_5)}')

El R^2 del Pipeline 5 es 0.5347516099607377


**Respuesta**:

Se logró obtener un R^2 de 0.535, lo cual en la práctica indica que la proporción de la variabilidad de la variable dependiente (sueldo) que es explicada por las variables independientes utilizadas, es del 53.5%. Lo cual es aceptable considerando que esta es una iteración inicial del modelo.

Para mejorar estos resultados se recomendaría considerar alguno de los siguientes puntos:

* Seleccionar y utilizar variables más relevantes del dataset, para reducir el número de características que no aportan información para el problema en cuestión y pueden estar afectando negativamente el rendimiento del modelo. 

* Experimentar ajustando los hiperparámetros de los modelos.

Con respecto a la pregunta de si se necesitarían más datos o otros tipos de características o una combinación de ambos para mejorar los resultados, es incierto, ya que primero se debería intentar maximizar el rendimiento del modelo con las características que ya se tienen, antes de conocer con exactitud si será necesario agregar otras extras.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media2.giphy.com/media/l1J3CbFgn5o7DGRuE/giphy.gif">
</p>

<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>